## Predicting Survival on the Titanic

### History
Perhaps one of the most infamous shipwrecks in history, the Titanic sank after colliding with an iceberg, killing 1502 out of 2224 people on board. Interestingly, by analysing the probability of survival based on few attributes like gender, age, and social status, we can make very accurate predictions on which passengers would survive. Some groups of people were more likely to survive than others, such as women, children, and the upper-class. Therefore, we can learn about the society priorities and privileges at the time.

### Assignment:

Build a Machine Learning Pipeline, to engineer the features in the data set and predict who is more likely to Survive the catastrophe.

Follow the Jupyter notebook below, and complete the missing bits of code, to achieve each one of the pipeline steps.

In [313]:
import re

# to handle datasets
import pandas as pd
import numpy as np

# for visualization
import matplotlib.pyplot as plt

# to divide train and test set
from sklearn.model_selection import train_test_split

# feature scaling
from sklearn.preprocessing import StandardScaler

# to build the models
from sklearn.linear_model import LogisticRegression

# to evaluate the models
from sklearn.metrics import accuracy_score, roc_auc_score

# to persist the model and the scaler
import joblib

# to visualise al the columns in the dataframe
pd.pandas.set_option('display.max_columns', None)

## Prepare the data set

In [314]:
# load the data - it is available open source and online

data = pd.read_csv('https://www.openml.org/data/get_csv/16826755/phpMYEkMl')

# display data
data.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29,0,0,24160,211.3375,B5,S,2,?,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.55,C22 C26,S,11,?,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2,1,2,113781,151.55,C22 C26,S,?,?,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30,1,2,113781,151.55,C22 C26,S,?,135,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25,1,2,113781,151.55,C22 C26,S,?,?,"Montreal, PQ / Chesterville, ON"


In [315]:
# replace interrogation marks by NaN values

data = data.replace('?', np.nan)

In [316]:
# retain only the first cabin if more than
# 1 are available per passenger

def get_first_cabin(row):
    try:
        return row.split()[0]
    except:
        return np.nan

data['cabin'] = data['cabin'].apply(get_first_cabin)

In [317]:
# extracts the title (Mr, Ms, etc) from the name variable

def get_title(passenger):
    line = passenger
    if re.search('Mrs', line):
        return 'Mrs'
    elif re.search('Mr', line):
        return 'Mr'
    elif re.search('Miss', line):
        return 'Miss'
    elif re.search('Master', line):
        return 'Master'
    else:
        return 'Other'

data['title'] = data['name'].apply(get_title)

In [318]:
# cast numerical variables as floats

data['fare'] = data['fare'].astype('float')
data['age'] = data['age'].astype('float')

In [319]:
# drop unnecessary variables

data.drop(labels=['name','ticket', 'boat', 'body','home.dest'], axis=1, inplace=True)

# display data
data.head()

,pclass,survived,sex,age,sibsp,parch,fare,cabin,embarked,title
0,1,1,female,29.0000,0,0,211.3375,B5,S,Miss
1,1,1,male,0.9167,1,2,151.5500,C22,S,Master
2,1,0,female,2.0000,1,2,151.5500,C22,S,Miss
3,1,0,male,30.0000,1,2,151.5500,C22,S,Mr
4,1,0,female,25.0000,1,2,151.5500,C22,S,Mrs


In [320]:
# save the data set

data.to_csv('titanic.csv', index=False)

## Data Exploration

### Find numerical and categorical variables

In [321]:
target = 'survived'

In [322]:
all_cols = data.columns
vars_num = ['pclass', 'sibsp', 'parch', 'age', 'fare']

vars_cat = [col for col in all_cols if col not in vars_num + [target]]

print('Number of numerical variables: {}'.format(len(vars_num)))
print('Number of categorical variables: {}'.format(len(vars_cat)))

Number of numerical variables: 5
Number of categorical variables: 4


### Find missing values in variables

In [323]:
# first in numerical variables

missing_id = data[vars_num].isna()

In [324]:
# now in categorical variables



### Determine cardinality of categorical variables

### Determine the distribution of numerical variables

## Separate data into train and test

Use the code below for reproducibility. Don't change it.

In [325]:
X_train, X_test, y_train, y_test = train_test_split(
    data.drop('survived', axis=1),  # predictors
    data['survived'],  # target
    test_size=0.2,  # percentage of obs in test set
    random_state=0)  # seed to ensure reproducibility

X_train.shape, X_test.shape

((1047, 9), (262, 9))

## Feature Engineering

### Extract only the letter (and drop the number) from the variable Cabin

In [326]:
def extract_letter(s: str) -> str | float:
    try:
        match = re.search("[a-zA-Z]", s)
        first_letter = match.group()
    except:
        first_letter = float('NaN')
    return f"cabin_{first_letter}"

X_train['cabin'] = X_train['cabin'].apply(extract_letter)
X_test['cabin'] = X_test['cabin'].apply(extract_letter)

### Fill in Missing data in numerical variables:

- Add a binary missing indicator
- Fill NA in original variable with the median

In [327]:
X_train

,pclass,sex,age,sibsp,parch,fare,cabin,embarked,title
1118,3,male,25.0000,0,0,7.9250,cabin_nan,S,Mr
44,1,female,41.0000,0,0,134.5000,cabin_E,C,Miss
1072,3,male,NaN,0,0,7.7333,cabin_nan,Q,Mr
1130,3,female,18.0000,0,0,7.7750,cabin_nan,S,Miss
574,2,male,29.0000,1,0,21.0000,cabin_nan,S,Mr
...,...,...,...,...,...,...,...,...,...
763,3,female,0.1667,1,2,20.5750,cabin_nan,S,Miss
835,3,male,NaN,0,0,8.0500,cabin_nan,S,Mr
1216,3,female,NaN,0,0,7.7333,cabin_nan,Q,Miss
559,2,female,20.0000,0,0,36.7500,cabin_nan,S,Miss


In [328]:
for col in vars_num:
    X_train[col + '_missing'] = X_train[col].isna().astype(int)
    X_train[col] = X_train[col].fillna(X_train[col].median())
    
    X_test[col + '_missing'] = X_test[col].isna().astype(int)
    X_test[col] = X_test[col].fillna(X_test[col].median())

In [329]:
X_train

,pclass,sex,age,sibsp,parch,fare,cabin,embarked,title,pclass_missing,sibsp_missing,parch_missing,age_missing,fare_missing
1118,3,male,25.0000,0,0,7.9250,cabin_nan,S,Mr,0,0,0,0,0
44,1,female,41.0000,0,0,134.5000,cabin_E,C,Miss,0,0,0,0,0
1072,3,male,28.0000,0,0,7.7333,cabin_nan,Q,Mr,0,0,0,1,0
1130,3,female,18.0000,0,0,7.7750,cabin_nan,S,Miss,0,0,0,0,0
574,2,male,29.0000,1,0,21.0000,cabin_nan,S,Mr,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,3,female,0.1667,1,2,20.5750,cabin_nan,S,Miss,0,0,0,0,0
835,3,male,28.0000,0,0,8.0500,cabin_nan,S,Mr,0,0,0,1,0
1216,3,female,28.0000,0,0,7.7333,cabin_nan,Q,Miss,0,0,0,1,0
559,2,female,20.0000,0,0,36.7500,cabin_nan,S,Miss,0,0,0,0,0


### Replace Missing data in categorical variables with the string **Missing**

In [330]:
for col in vars_cat:
    X_train[col] = X_train[col].fillna('Missing')
    X_test[col] = X_test[col].fillna('Missing')

### Remove rare labels in categorical variables

- remove labels present in less than 5 % of the passengers

In [331]:
X_train

,pclass,sex,age,sibsp,parch,fare,cabin,embarked,title,pclass_missing,sibsp_missing,parch_missing,age_missing,fare_missing
1118,3,male,25.0000,0,0,7.9250,cabin_nan,S,Mr,0,0,0,0,0
44,1,female,41.0000,0,0,134.5000,cabin_E,C,Miss,0,0,0,0,0
1072,3,male,28.0000,0,0,7.7333,cabin_nan,Q,Mr,0,0,0,1,0
1130,3,female,18.0000,0,0,7.7750,cabin_nan,S,Miss,0,0,0,0,0
574,2,male,29.0000,1,0,21.0000,cabin_nan,S,Mr,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,3,female,0.1667,1,2,20.5750,cabin_nan,S,Miss,0,0,0,0,0
835,3,male,28.0000,0,0,8.0500,cabin_nan,S,Mr,0,0,0,1,0
1216,3,female,28.0000,0,0,7.7333,cabin_nan,Q,Miss,0,0,0,1,0
559,2,female,20.0000,0,0,36.7500,cabin_nan,S,Miss,0,0,0,0,0


In [332]:
n = len(X_train)
thres = 0.05

In [333]:
for col in vars_cat:
    rare_labels = X_train[col].value_counts()[X_train[col].value_counts()/n < thres].index
    X_train[col] = X_train[col].replace(rare_labels, f'{col}_rare')
    
    rare_labels = X_test[col].value_counts()[X_test[col].value_counts()/n < thres].index
    X_test[col] = X_test[col].replace(rare_labels, f'{col}_rare')


In [336]:
X_train

,pclass,sex,age,sibsp,parch,fare,cabin,embarked,title,pclass_missing,sibsp_missing,parch_missing,age_missing,fare_missing
1118,3,male,25.0000,0,0,7.9250,cabin_nan,S,Mr,0,0,0,0,0
44,1,female,41.0000,0,0,134.5000,cabin_rare,C,Miss,0,0,0,0,0
1072,3,male,28.0000,0,0,7.7333,cabin_nan,Q,Mr,0,0,0,1,0
1130,3,female,18.0000,0,0,7.7750,cabin_nan,S,Miss,0,0,0,0,0
574,2,male,29.0000,1,0,21.0000,cabin_nan,S,Mr,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,3,female,0.1667,1,2,20.5750,cabin_nan,S,Miss,0,0,0,0,0
835,3,male,28.0000,0,0,8.0500,cabin_nan,S,Mr,0,0,0,1,0
1216,3,female,28.0000,0,0,7.7333,cabin_nan,Q,Miss,0,0,0,1,0
559,2,female,20.0000,0,0,36.7500,cabin_nan,S,Miss,0,0,0,0,0


### Perform one hot encoding of categorical variables into k-1 binary variables

- k-1, means that if the variable contains 9 different categories, we create 8 different binary variables
- Remember to drop the original categorical variable (the one with the strings) after the encoding

In [337]:
import pandas as pd
for col in vars_cat:
    X_train = pd.concat([X_train.drop(columns=col), pd.get_dummies(X_train[col])], axis=1)
    X_test = pd.concat([X_test.drop(columns=col), pd.get_dummies(X_test[col])], axis=1)


In [347]:
X_test

,pclass,age,sibsp,parch,fare,pclass_missing,sibsp_missing,parch_missing,age_missing,fare_missing,female,male,cabin_nan,cabin_rare,C,S,embarked_rare,Mr,title_rare
1139,0.867071,0.668521,-0.506817,-0.409606,-0.436773,0,0,0,0,0,False,True,True,False,False,True,False,True,False
533,-0.347756,-0.526380,-0.506817,0.652936,-0.194311,0,0,0,0,0,True,False,True,False,False,True,False,False,True
459,-0.347756,0.949674,0.769972,-0.409606,-0.083295,0,0,0,0,0,False,True,True,False,False,True,False,True,False
1150,0.867071,-0.174938,-0.506817,-0.409606,-0.314578,0,0,0,1,0,False,True,True,False,False,True,False,True,False
393,-0.347756,-0.245227,-0.506817,-0.409606,-0.000033,0,0,0,0,0,False,True,True,False,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
753,0.867071,-0.456091,-0.506817,-0.409606,-0.433920,0,0,0,0,0,False,True,True,False,False,True,False,True,False
1052,0.867071,-0.174938,-0.506817,-0.409606,-0.436773,0,0,0,1,0,False,True,True,False,False,True,False,True,False
426,-0.347756,0.106215,-0.506817,-0.409606,-0.342332,0,0,0,0,0,False,True,True,False,False,True,False,True,False
554,-0.347756,-0.174938,-0.506817,-0.409606,-0.342332,0,0,0,0,0,False,True,True,False,False,True,False,True,False


In [351]:
# False since we know that the new columns in test are false for all (otherwise a dummy would have already existed)
X_test = X_test.reindex(columns=X_train.columns, fill_value=False)

### Scale the variables

- Use the standard scaler from Scikit-learn

In [350]:
X_test

,pclass,age,sibsp,parch,fare,pclass_missing,sibsp_missing,parch_missing,age_missing,fare_missing,female,male,cabin_nan,cabin_rare,C,S,embarked_rare,Mr,title_rare
1139,0.867071,0.668521,-0.506817,-0.409606,-0.436773,0,0,0,0,0,False,True,True,False,False,True,False,True,False
533,-0.347756,-0.526380,-0.506817,0.652936,-0.194311,0,0,0,0,0,True,False,True,False,False,True,False,False,True
459,-0.347756,0.949674,0.769972,-0.409606,-0.083295,0,0,0,0,0,False,True,True,False,False,True,False,True,False
1150,0.867071,-0.174938,-0.506817,-0.409606,-0.314578,0,0,0,1,0,False,True,True,False,False,True,False,True,False
393,-0.347756,-0.245227,-0.506817,-0.409606,-0.000033,0,0,0,0,0,False,True,True,False,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
753,0.867071,-0.456091,-0.506817,-0.409606,-0.433920,0,0,0,0,0,False,True,True,False,False,True,False,True,False
1052,0.867071,-0.174938,-0.506817,-0.409606,-0.436773,0,0,0,1,0,False,True,True,False,False,True,False,True,False
426,-0.347756,0.106215,-0.506817,-0.409606,-0.342332,0,0,0,0,0,False,True,True,False,False,True,False,True,False
554,-0.347756,-0.174938,-0.506817,-0.409606,-0.342332,0,0,0,0,0,False,True,True,False,False,True,False,True,False


In [352]:
scaler = StandardScaler()

# Fit and transform the data
X_train[vars_num] = scaler.fit_transform(X_train[vars_num])
X_test[vars_num] = scaler.fit_transform(X_test[vars_num])
# X_test

## Train the Logistic Regression model

- Set the regularization parameter to 0.0005
- Set the seed to 0

In [353]:
lr = LogisticRegression(C=0.0005, random_state=0)
mod = lr.fit(X_train, y_train)

y_pred_probs = mod.predict_proba(X_test)

## Make predictions and evaluate model performance

Determine:
- roc-auc
- accuracy

**Important, remember that to determine the accuracy, you need the outcome 0, 1, referring to survived or not. But to determine the roc-auc you need the probability of survival.**

In [354]:
auc = roc_auc_score(y_test, y_pred_probs[:,1])

y_pred = np.argmax(y_pred_probs, axis=1)
accuracy = accuracy_score(y_test, y_pred)


print(f'AUC: {auc:.2f}')
print(f'Accuracy: {accuracy:.2f}')


ValueError: Found input variables with inconsistent numbers of samples: [1047, 262]

That's it! Well done

**Keep this code safe, as we will use this notebook later on, to build production code, in our next assignement!!**